<a href="https://colab.research.google.com/github/EduardoArion/mlops-repo/blob/feature-gobcdmx/Catastral_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Archivo Catastral

En su momento subí directamente los archivos a google colab, y desde el historial de uploads los llamé para hacer el concat, deberé modificar este pedazo de código una vez establezcamos donde almacenaremos lo datos

In [ ]:
catastral = list(uploaded.keys())
print(uploaded.keys())


dict_keys(['catastro2021_ALVARO_OBREGON.csv', 'catastro2021_AZCAPOTZALCO.csv', 'catastro2021_BENITO_JUAREZ.csv', 'catastro2021_COYOACAN.csv', 'catastro2021_CUAJIMALPA.csv', 'catastro2021_CUAUHTEMOC.csv', 'catastro2021_GUSTAVO_A_MADERO.csv', 'catastro2021_IZTACALCO.csv', 'catastro2021_IZTAPALAPA.csv', 'catastro2021_MAGDALENA_CONTRERAS.csv', 'catastro2021_MIGUEL_HIDALGO.csv', 'catastro2021_MILPA_ALTA.csv', 'catastro2021_TLAHUAC.csv', 'catastro2021_TLALPAN.csv', 'catastro2021_VENUSTIANO_CARRANZA.csv', 'catastro2021_XOCHIMILCO.csv'])


In [ ]:
df_catastral = [pd.read_csv(nombre) for nombre in catastral]
df_final_catastral = pd.concat(df_catastral, axis=0, ignore_index=True)

/tmp/ipython-input-2182653453.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_catastral = [pd.read_csv(nombre) for nombre in catastral]


In [ ]:
# Guardar en CSV
df_final_catastral.to_csv("concat_catastral.csv", index=False)

# Descargar
files.download("concat_catastral.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Voy a verificar que no haya peridido filas en el concat y asi fue, no perdí ni una sola
Total filas en todos los archivos: 2295465
Filas en df_final: 2295465

In [ ]:
import pandas as pd

# Supongamos que ya tienes la lista de nombres
archivos = list(uploaded.keys())

# Leer y contar filas de cada archivo
total_filas = 0
for archivo in archivos:
    df = pd.read_csv(archivo)
    filas = len(df)
    total_filas += filas
    print(f"{archivo}: {filas} filas")

print("Total filas en todos los archivos:", total_filas)

# Ahora cuentas en el df_final
print("Filas en df_final:", len(df_final_catastral))


/tmp/ipython-input-4063968483.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(archivo)


catastro2021_ALVARO_OBREGON.csv: 180377 filas
catastro2021_AZCAPOTZALCO.csv: 113617 filas
catastro2021_BENITO_JUAREZ.csv: 216309 filas
catastro2021_COYOACAN.csv: 168159 filas
catastro2021_CUAJIMALPA.csv: 52753 filas
catastro2021_CUAUHTEMOC.csv: 246722 filas
catastro2021_GUSTAVO_A_MADERO.csv: 245885 filas
catastro2021_IZTACALCO.csv: 99684 filas
catastro2021_IZTAPALAPA.csv: 366324 filas
catastro2021_MAGDALENA_CONTRERAS.csv: 38033 filas
catastro2021_MIGUEL_HIDALGO.csv: 170338 filas
catastro2021_MILPA_ALTA.csv: 10216 filas
catastro2021_TLAHUAC.csv: 67652 filas
catastro2021_TLALPAN.csv: 139958 filas
catastro2021_VENUSTIANO_CARRANZA.csv: 120801 filas
catastro2021_XOCHIMILCO.csv: 58637 filas
Total filas en todos los archivos: 2295465
Filas en df_final: 2295465


Ahora haré un merge de mis dos archivos, concat_catastral.csv y AGEB_fm

Lo que noto es que al intentar hacer el merge con nombres voy a tener problemas con las calles que sea del estilo
calle: "Sur 170", número: 7 --> Sur 170 7
Sostenes Rocha Gral 73 a
Sostenes Rocha S/n Esq. Sur 122
SUR 112 S N
Sur 112 a Num 19
Y demás ejemplos, es como aparece en mis datos, entonces creo que lo que deberé hacer es separar el nombre del numero por el primer espacio que haya de derecha a izquierda para así generar una sola columna que me de solo el nombre y esta la pueda unir a

NOMVIAL en AGEB_fm

Yo se que de este manera perderé datos si intento hacer un match exacto, por lo que tendré que hacer un merge con aproximaciones del nombre, por ejemplo con un startswith()

tambien voy a agregar una etiqueta ("_cat") a los datos de concat_catastral que se sumen a AGEB_fm



Primero le daré tratamiento al df de catastral para crear el nombre de la calle, el cual sera mi llave de merge

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/TFM/concat_catastral.csv" /content/
df_catastral = pd.read_csv("concat_catastral.csv")  # ahora carga más rápido

!cp "/content/drive/MyDrive/TFM/AGEB_fm.csv" /content/
df_AGEB_fm = pd.read_csv("AGEB_fm.csv")  # ahora carga más rápido


/tmp/ipython-input-3051520690.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_catastral = pd.read_csv("concat_catastral.csv")  # ahora carga más rápido
/tmp/ipython-input-3051520690.py:5: DtypeWarning: Columns (6,50,104,111,112,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df_AGEB_fm = pd.read_csv("AGEB_fm.csv")  # ahora carga más rápido


In [ ]:
df_catastral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2295465 entries, 0 to 2295464
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   fid                   float64
 1   fid_2                 float64
 2   calle_numero          object 
 3   codigo_postal         object 
 4   colonia               object 
 5   alcaldia              object 
 6   sup_terreno           float64
 7   sup_construccion      float64
 8   anio_construccion     float64
 9   instal_esp            int64  
 10  valor_unitario_suelo  float64
 11  valor_suelo           float64
 12  cve_vus               object 
 13  subsidio              float64
dtypes: float64(8), int64(1), object(5)
memory usage: 245.2+ MB


In [ ]:
df_catastral["calle_numero"] = df_catastral["calle_numero"].astype(str)

df_catastral["calle_numero"] = df_catastral["calle_numero"].fillna("").astype(str)

In [ ]:
type(df_catastral["calle_numero"].iloc[0])

str

In [ ]:
"""
import re
import unicodedata

def normalizar_calle(texto: str) -> str:
    """
    Normaliza nombres de calles eliminando ruido común (Num, Dpto, Edif, Casa, etc.)
    y conservando solo el nombre principal y número base de la calle.
    """

    if not texto or not isinstance(texto, str):
        return ""

    # --- 1. Normalización básica ---
    # Pasar a mayúsculas
    t = texto.upper()

    # Quitar acentos y caracteres raros
    t = ''.join(
        c for c in unicodedata.normalize('NFKD', t)
        if not unicodedata.combining(c)
    )

    # Uniformizar abreviaturas comunes
    reemplazos = {
        r"\bCL\.?\b": "CALLE",
        r"\bCLL\.?\b": "CALLE",
        r"\bAVD?\.?\b": "AVENIDA",
        r"\bAV\.?\b": "AVENIDA",
        r"\bCD?A\.?\b": "CDA",
        r"\bCALZ\.?\b": "CALZADA",
        r"\bEJE\b": "EJE",
        r"\bPTE\b": "PONIENTE",
        r"\bNTE\b": "NORTE",
        r"\bOTE\b": "ORIENTE",
        r"\bSUR\b": "SUR",
        r"\bDR\.?\b": "DOCTOR",
        r"\bGRAL\.?\b": "GENERAL",
        r"\bING\.?\b": "INGENIERO",
    }
    for patron, repl in reemplazos.items():
        t = re.sub(patron, repl, t)

    # --- 2. Eliminar ruido ---
    ruido = [
        r"\bNUM(ERO)?\.?\b",
        r"\bNO\.?\b",
        r"\bN\.?\b",
        r"\bDEP(TO)?\.?\b",
        r"\bDEPARTAMENTO\b",
        r"\bCASA\b",
        r"\bED(IF)?\.?\b",
        r"\bEDIFICIO\b",
        r"\bTORRE\b",
        r"\bCOND(OMINIO)?\b",
        r"\bCONJ(UNTO)?\b",
        r"\bSECC?\.?\b",
        r"\bSEC\.?\b",
        r"\bMZ\.?\b",
        r"\bMANZANA\b",
        r"\bLT\.?\b",
        r"\bLOTE\b",
        r"\bINT\.?\b",
        r"\bINTERIOR\b",
        r"\bENT\.?\b",
        r"\bAND\.?\b",
        r"\bGRUPO\b",
        r"\bAGRUP\b",
        r"\bPASEO\b",
        r"\bEST(AC)?\.?\b",
        r"\b#",
    ]
    for patron in ruido:
        t = re.sub(patron, "", t)

    # --- 3. Limpieza de caracteres ---
    t = re.sub(r"[^A-Z0-9\s]", " ", t)  # quitar símbolos
    t = re.sub(r"\s+", " ", t)          # espacios extra
    t = t.strip()

    # --- 4. Correcciones contextuales ---
    # Quitar letras sueltas al final de "SUR 114 A" → "SUR 114"
    t = re.sub(r"\b(SUR|NORTE|ORIENTE|PONIENTE|EJE \d+)\s+(\d+)[A-Z]\b", r"\1 \2", t)

    # "CALLE 6" sin número → mantener "CALLE 6"
    # "CALLE 6, 106" → "CALLE 6 106"
    t = re.sub(r"\bCALLE\s+(\d+)\s*,?\s*(\d+)", r"CALLE \1 \2", t)

    # --- 5. Output final ---
    return t

"""

In [ ]:
# Aplicar la función a la columna calle_numero
df_catastral["calle_norm"] = df_catastral["calle_numero"].apply(normalizar_calle)


In [ ]:
df_catastral.head(1000)

,fid,fid_2,calle_numero,codigo_postal,colonia,alcaldia,sup_terreno,sup_construccion,anio_construccion,instal_esp,valor_unitario_suelo,valor_suelo,cve_vus,subsidio,calle_norm
0,1.0,1966352.0,Observatorio 297,1120.0,Cove,ALVARO OBREGON,159.58,249.35,2001.0,1,2604.93,415694.7294,A010513,644.260919,OBSERVATORIO 297
1,2.0,1966365.0,Observatorio 299,1120.0,Cove,ALVARO OBREGON,171.81,252.26,1969.0,0,2604.93,447553.0233,A010513,746.544515,OBSERVATORIO 299
2,3.0,1966359.0,Poniente 79 67,1120.0,Cove,ALVARO OBREGON,180.00,141.00,1970.0,0,2604.93,468887.4000,A010513,643.418583,PONIENTE 79
3,4.0,96480.0,Observatorio 301,1120.0,Cove,ALVARO OBREGON,176.31,28.58,1970.0,1,2604.93,459275.2083,A010513,615.591443,OBSERVATORIO 301
4,5.0,1966363.0,Observatorio 305,1120.0,Cove,ALVARO OBREGON,200.00,337.00,1997.0,0,2604.93,520986.0000,A010513,593.403316,OBSERVATORIO 305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,540.0,2518111.0,"And. ESTRELLA DE BELEN, MZ-A LT-7",1220.0,LA ESTRELLA,ALVARO OBREGON,73.66,173.38,2009.0,1,2355.49,173505.3934,A010541,698.719263,AND ESTRELLA DE BELEN MZ-A LT-7
996,541.0,2521327.0,AV VASCO DE QUIROGA M-F L-9,1220.0,CUEVITAS,ALVARO OBREGON,83.00,142.45,2004.0,0,2355.49,195505.6700,A010541,566.796447,AV VASCO DE QUIROGA M-F L-9
997,542.0,2521515.0,ESTRELLA FUGAZ M-E JTO13,1220.0,CUEVITAS,ALVARO OBREGON,89.00,177.40,1996.0,0,2355.49,209638.6100,A010541,668.010590,ESTRELLA FUGAZ M-E JTO13
998,543.0,2521305.0,Andador Estrella Blanca Mz.g Lt.31,1210.0,Estrella,ALVARO OBREGON,84.00,14.00,1996.0,0,2355.49,197861.1600,A010541,197.150504,ANDADOR ESTRELLA BLANCA MZ G LT 31


In [ ]:
# Mostrar el texto completo en cada columna (sin cortar)
pd.set_option("display.max_colwidth", None)

# (Opcional) Si además quieres ver todas las filas y columnas:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)


#print(df_catastral[["calle_numero", "calle_norm"]].head(1000).to_string())
print(df_catastral[["calle_numero", "calle_norm"]].sample(n=1000, random_state=200).to_string())

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option("display.max_colwidth")


Eje 5 SUR AV SAN ANTONIO, 455 Edif. STAR SAN ANTONIO II, Dpto. 816                                                    EJE 5
EJE 1 PONIENTE GRO. N. 202  Est. 28                                                    EJE 1
EJE 7-A SUR GRAL. EMILIANO ZAPATA 252 DEPTO 414                                                    EJE 7
Eje 1 PONIENTE CALZ VALLEJO, 1268 Edif. E-4, Sec. COND 4, Dpto. 102                                                    EJE 1
Eje 6 SUR INDEPENDENCIA, 176 PH 2                                                    EJE 6
EJE 2 OTE CALZADA DE LA VIGA, 996 Casa H-13                                                    EJE 2
EJE 2 OTE CALZADA DE LA VIGA, 369 Dpto. 608                                                    EJE 2
EJE 2 OTE CALZADA DE LA VIGA, 1225 Dpto. 207                                                    EJE 2
Eje 5 SUR AV SAN ANTONIO, 455 Edif. STAR SAN ANTONIO II, Dpto. 816                                                    EJE 5
Avd. LOMAS DE PLATEROS, 11 Sec. ESTAC 4, Cjon. 30                                                  AVENIDA
Eje 1 PONIENTE CALZ VALLEJO, 1268 Edif. E-4, Sec. COND 4, Dpto. 102                                                    EJE 1
Eje 6 SUR INDEPENDENCIA, 176 PH 2                                                    EJE 6
AV SANTA FE 459 DEPTO 2204                                                  AVENIDA


                                                                   calle_numero                                               calle_norm
1184479                                           Trebol Mz 2 Lt 20 B Pradera I                                                   TREBOL
1914272                                              AV.RENO 40 EDIF.B D.T. 302                                          AVENIDA RENO 40
84343                                                 Cerrada de Hidalgo Num.14                                CERRADA DE HIDALGO NUM 14
802923                                                Tuxpan 6 Oficina 201 Roma                                TUXPAN 6 OFICINA 201 ROMA
795716                                                                  Tula 31                                                  TULA 31
1082391                                                          Pernambuco 891                                           PERNAMBUCO 891
1412341                                  

OUTPUT        ORIGINAL        WANTED OUTPUT
CL. SUR 128     Cl. SUR 128, 12       SUR 128
SUR 126 A.       Sur 126 a 9      SUR 126
SUR 126-A NUM.      Sur 126-a Num.17      SUR 126
CL. SUR 128, 22 CASA      Cl. SUR 128, 22 Casa 4    SUR 128
CL. SUR 128, 22 DPTO.     Cl. SUR 128, 22 Dpto. 1   SUR 128
SUR         SUR 122 S/N     SUR 122
SUR 122 ESQ PONIENTE        Sur 122 Esq Poniente 83     SUR 122
SUR 112 A 23 DEPTO        Sur 112 a 23 Depto 12        SUR 112
SUR 120 A-6 DEPTO       Sur 120 A-6 Depto 302-C       SUR 120
CL. SUR 124, 3 DPTO.          Cl. SUR 124, 3 Dpto. 3        SUR 124
SUR 114 NUM. 46  DEPTO.       SUR 114 N. 46  Depto. 303      SUR 114

Sostenes Rocha 18                                    ROCHA 18       SOSTENES ROCHA 18
Gral. Felix u Gomez 95                                    GOMEZ 95      GRAL FELIX U GOMEZ 95
Sostenes Rocha Gral 10                                     GRAL 10      SOSTENES ROCHA GRAL
Cda. 3 de Observatorio Nu.10 Depto-302                      CDA 3       OBSERVATORIO 10
Cl. GRAL MANUEL M PLATA, 123                             PLATA 123      GRAL MANUEL M PLATA 123
Plata Gral 110                                    GRAL 110          PLATA GRAL 110
Cl. CAMINO A BELEN, 85 Edif. R, Casa 223                 BELEN 85       CAMINO A BELEN 85
Vereda del Tanque 48 Lomas de Santa Fe                 TANQUE 48        VEREDA DEL TANQUE 48
Calle Estrella Fugas Mz.c Lt.13                       LT 13             ESTRELLA FUGAZ
Prv. RAUL ZARATE MACHUCA, 11                          MACHUCA 11        RAUL ZARATE MACHUCA 11



Cl. SUR 124, 3 Dpto. 4                   DPTO 4           SUR 124
Cl. SUR 114 A, 21 BIS Dpto. 4            BIS DPTO 4       SUR 114
Sur 114-a 29                             A 29             SUR 114
AV. OBSERVATORIO  209  INT.02            INT 02           OBSERVATORIO 209
Cl. SUR 114, 29 Dpto. 1                   DPTO 1          SUR 114
Sur 114 a 22                            A 22              SUR 114
Sur 116 34 Dpto.  4                      DPTO 4           SUR 114
Sur 114 a 43 Dpto.  301                     DPTO 301      SUR 114
Avd. OBSERVATORIO, 151 Dpto. 01-A           DPTO 01-A     OBSERVATORIO 151




Cda. 3 de Obesrvatorio Num.10 Depto-201                                          CDA 3    Cda. 3 de Obesrvatorio 10
Cda. 4A CDA OBSERVATORIO, 29                       CDA 4    Cda. 4A CDA OBSERVATORIO 29
Eje 3 PONIENTE AMORES, 33 Dpto.S202                                               EJE 3    EJE 3 33
Eje 3 Nte San Isidro 368 Edif D Depto 204                                                EJE 3    EJE 368
OTE 259 No 108 ED 5 TURQUESA DEPTO 011                                                OTE 259    OTE 259 108
COYOACAN EJE 3 PTE N. 33  Depto. 403                                         COYOACAN EJE 3      EJE 3 33
Calle 17 a Mz 3 a Lt 01                           A MZ 3     CALLE 17
EJE 2 OTE CALZADA DE LA VIGA, 996 Casa A-5                                                  EJE 2    EJE 2 996
Eje 1 PONIENTE CALZ VALLEJO, 1268 COND 3 EDIF H13 DEPTO 403                                             EJE 1     EJE 1 1268
Calle 11 Num 121 Casa 18                    NUM 121       CALLE 11 121
Calle 2 42 Estac 45                          ESTAC 45     CALLE 2 42
Calle 29 Num 51 Depto a 201               NUM 51           CALLE 29 51
Calle 13 170                              13 170           CALLE 13 170
Calle 631 Num. 17                        NUM 17            CALLE 631
Norte 178 566                         NORTE 178            NORTE 178 566
Sur 16 233                                SUR 16           SUR 16 233
Eje 1 OTE FC HIDALGO, 32 Edif. B2, Dpto. B-111                                 EJE 1         EJE 1 32
Calle 4 Num 222 Depto 102 Edif B                                           NUM 222            CALLE 4 222
Calle 327 708                          327 708       CALLE 327 708
Norte 1-j Num 4909                     NORTE 1        NORTE 1 4909
Eje 8 SUR AV POPOCATEPETL, 510 Edif. D, Dpto. 202                                    EJE 8      EJE 8 510
Eje 1 NORTE JOSE ANTONIO ALZATE, 165 Y 169 Estac. 5                                       EJE 1       EJE 1 165
Oriente 247 a No 192 Edif 02 Depto 103                                       ORIENTE 247            ORIENTE 247 192
Tasque@a 110                    A 110           TASQUE@A 110
Ruise@or 62                     OR 62           RUISE@OR 62
Av Canal De Tezontle S/N Cond 2 Edif C Depto 204                               NUM COND 2       CANAL DE TEZONTLE
Pe#oles Num. 47 Edif. a Depto. 502          OLES NUM 47       PE#OLES 47
Norte 66 Num 3740 Depto B 101               NORTE 66         NORTE 66 3740
AV 11 402 COND I EDIF A DEPTO 404          AV 11            AV 11 402
Av 699 110 Agrup D Casa 8                 AV 699             AV 699 110
Coru#a 235 Depto. 103                     A 235               CORU#A 235
Av 565 32                               AV 565               AV 565 32
Av 11 Num 62 Conj. 13 Casa 36             AV 11               AV 11 62
Tasque@a 1818 Grupo D Casa 74 Paseos de
Tasque@a                                   A 1818       TASQUE@A 1818



 Isabel La Catolica 134 Edif C Depto 03      ISABEL LA CATOLICA 134 C 03      ISABEL LA CATOLICA 134
 Cl. DR NAVARRO, 60 Edif. A, Ent. M-2, Dpto. 303    CALLE DR NAVARRO 60 A ENT M 2 DPTO 303    CALLE DR NAVARRO 60
Rancho San Lorenzo 81 Depto 302 a      RANCHO SAN LORENZO 81 302 A          RANCHO SAN LORENZO 81
 Cl. DR JOSE MARIA VERTIZ, 101 DEPTO.407      CALLE DR JOSE MARIA VERTIZ 101 407      CALLE DR JOSE MARIA VERTIZ 101
 IGNACIO ALDAMA, 188 Sec. MZ-5, CASA 28         IGNACIO ALDAMA 188 SEC MZ 5 28        IGNACIO ALDAMA 188
 JUAREZ 63 CASA 9                              JUAREZ 63 9              JUAREZ 63
 Calle 21 22 Depto. 1                         CALLE 21 22 1             CALLE 21 22
Av 11 Num 62 Conj. 19 Casa 44              AV 11 62 19 44               AV 11 62
Cipres 380 Edif. G Depto. 205              CIPRES 380 G 205             CIPRES 380





Isabel La Catolica 134 Edif C Depto 03                    ISABEL LA CATOLICA 134 C 03         ISABEL LA CATOLICA 134
Cl. DR NAVARRO, 60 Edif. A, Ent. M-2, Dpto. 303           CALLE DOCTOR NAVARRO 60 A M 2 DPTO 303      CALLE DOCTOR NAVARRO 60
Cl. LA MORENA, 425 Dpto. 401                              CALLE LA MORENA 425 DPTO 401          CALLE LA MORENA 425
Av. Tamaulipas 1279 Edif 8 Depto 002                       AV TAMAULIPAS 1279 002             AV TAMAULIPAS 1279
Nueva York 33 Depto B 306                                  NUEVA YORK 33 B 306                NUEVA YORK 33
esus Carranza 21 e 108                                    JESUS CARRANZA 21 E 108             JESUS CARRANZA 21
Miguel Laurent 1710 Dpto.  202                            MIGUEL LAURENT 1710 DPTO 202         MIGUEL LAURENT 1710
Cl. TECNICOS Y MANUALES, 200 Edif. 15, Dpto. 401         CALLE TECNICOS Y MANUALES 200 DPTO 401       CALLE TECNICOS Y MANUALES 200




CALLE 2, 270 Edif. B, Dpto. 501                       CALLE 2 DPTO 501                    CALLE 2 270
Agua Caliente 129 Edif 2 Depto 501                     AGUA CALIENTE 129 501              AGUA CALIENTE 129
Cl. LAGO GUIJA, 72 Dpto. 710                           CALLE LAGO GUIJA 72 DPTO 710       CALLE LAGO GUIJA 72
 Cl. SIERRA GORDA, 23 CONJ ENCINOS TORRE G D 10        CALLE SIERRA GORDA 23 ENCINOS TORRE G D 10         CALLE SIERRA GORDA 23
IGNACIO ALDAMA, 188 Sec. MZ-5, CASA 28                 IGNACIO ALDAMA 188 28                IGNACIO ALDAMA 188
Calle 602 184                                          CALLE 602                          CALLE 602 184
Calle 2 Num 188 Estac 64                               CALLE 2 64                         CALLE 2 188
ALZADA VALLEJO 1307 EST.35                             CALZADA VALLEJO 1307 EST 35         CALZADA VALLEJO 1307
Cl. TENOCHTITLAN, 100 Edif. D-10, Dpto. 302            CALLE TENOCHTITLAN 100 DPTO 302       CALLE TENOCHTITLAN 100
Paseo de la Hacienda 79 Depto 401 Paseo de Taxque       PASEO DE LA HACIENDA 79 PASEO DE TAXQUE       PASEO DE LA HACIENDA 79
Calle 21 22 Depto. 1                                    CALLE 21 1                        CALLE 21 22
MONROVIA 806 DEPTO GH 04                                MONROVIA 806 GH 04                MONROVIA 806
Calle 26 118                                            CALLE 26                          CALLE 26 118
Calle 35 Num 35                                          CALLE 35                         CALLE 35 35
MIGUEL BERNARD, 399 Edif. B, Dpto. 36                   MIGUEL BERNARD 399 DPTO 36        MIGUEL BERNARD 399



Sur 121 529                                               SUR 121529          SUR 121 529       
Cl. 1 DE MAYO, 114 Dpto. B304                             CALLE 1MAYO 114B304         1 DE MAYO 114
EJE 2 NORTE EULALIA GUZMAN, 150 Dpto. 205                 EJE 2150          EJE 2 150
Cl. EUGENIO SUE, 19 Edif. EUGENIO SUE, Dpto. 310          DPTO 310          EUGENIO SUE 19
Av 537 203                                                AV 537          AV 537 203
Calz. VALLEJO, 122 Edif. TORRE A, Dpto. 102               CALZADA VALLEJO 122DPTO 102         CALZADA VALLEJO 122
Calle 623 218                                             CALLE 623         CALLE 623 218
Pte 106 Num 508 Col Defensores de la Rep                  PONIENTE 106508DEFENSORES DE LA REP         PONIENTE 106 508
Pico de Pe#a Lara 53 Depto.1001                           A LARA 53         PICO DE PE#A LARA 53
Cl. SERRANILLAS, 166-a Edif. C-6, Sec. LT-2, Dpto. 004             DPTO 004         SERRANILLAS 166
5 de Mayo # 61 Edif. a Depto. 304                         5MAYO # 61          5 DE MAYO 61
Calle 699 92                                              CALLE 699         Calle 699 92
Calz. AZCAPOTZALCO LA VILLA, 170 Edif. B, Dpto. 314               CALZADA AZCAPOTZALCO LA VILLA 170DPTO 314         CALZADA AZCAPOTZALCO LA VILLA 170
Avenida 11 402 Edif.5 Estac.27                            AVENIDA 11            AVENIDA 11 402
Avd. MEXICO, 363 Edif. TORRE C, Dpto. C-703               C 703           AVENIDA MEXICO 363
Villa Hermosa 79 Secc. F Mz. C Lt. 40                     C LT 40         VILLA HERMOSA 79
Cl. 6, 106 Edif. COND 1-B EDIF 7, Dpto. 403               CALLE 6         CALLE 6 106
Avd. RICARDO FLORES MAGON, 74 And. ALMACENES 74, Edif. 338 CHIHUAHUA, Ent. B, Dpto. 1308       DPTO 1308       RICARDO FLORES MAGON 74   
Cl. LAGO HIELMAR, 44 Edif. G, Dpto. G-703                 G 703         LAGO HIELMAR 44
Av 11 Num 62 Conj. 14 Casa 18                             AV 11         AV 11 62
Calle Canela 19 Ed B Dep 103                              DEP 10          CANELA 19
Cerro del Pe@On 133                                       ON 133          Cerro del Pe@On 133

    
Ignacio Comonfort 168 F-205                      IGNACIO COMONFORT 168 F 205          IGNACIO COMONFORT 168
Avd. PACIFICO, 473 Dpto. 411                      AVENIDA PACIFICO 473 DPTO 411         AVENIDA PACIFICO 473
Callejon de Minas 27 Casa 8                      CALLEJON DE MINAS 27 8           CALLEJON DE MINAS 27
Dr Martinez del Rio 171 B 201                   DOCTOR MARTINEZ DEL RIO 171 B 201         DOCTOR MARTINEZ DEL RIO 171
Heroes 224 Depto 301 B                           HEROES 224 301 B         HEROES 224
Carril 13 Edif 30 Depto 302                      CARRIL 13 30 302         CARRIL 13
Cl. PALMA, 204 Edif. B, Dpto. B-202              CALLE PALMA 204 B DPTO B 202         CALLE PALMA 204
Republica de Chile 49 Depto G 103                 REPUBLICA DE CHILE 49 G 103         REPUBLICA DE CHILE 49
Gavilan 181 Edif.4-d Depto.101                    GAVILAN 181 4 D 101           GAVILAN 181


Eje 2 A SUR SAN LUIS POTOSI, 123                        EJE 2      EJE 2 123
Tuxpan 48 Roma Sur                                     SUR         TUXPAN 48
Lazaro Cardenas S/n Cda                                CD          LAZARO CARDENAS
Eje 5 OTE JAVIER ROJO GOMEZ, 462                        EJE 5      EJE 5 462
Division del Norte 3159 Depto C 9                      NORTE 3159     DIVISION DEL NORTE 3159
Eje 3 PTE AV THIERS, 251 Ofi. 1101                     EJE 3       EJE 3 251
Insurgentes Sur 1388 Ofic 604                           SUR 1388      INSURGENTES SUR 1388
5 DE FEBRERO 523 DEPTO. A-402                          5          5 DE FEBRERO 523
Prolongacion 5 de Mayo 3050 Casa 407                  PROLONGACION 5      PROLONGACION 5 DE MAYO 3050
Eje 1 PONIENTE CALZ VALLEJO, 1268 Edif. G-5, Sec. COND 2, Dpto. 102             EJE 1       EJE 1 1268
16 de Septiembre 31                                    16            16 DE SEPTIEMBRE 31
2o de Noviembre 445 Edif. e Depto. 404                 2            20 DE NOVIEMBRE 445
Hermosillo 7 Depto 302 Roma Sur                        SUR          HERMOSILLO 7
EJE 2 PONIENTE GABRIEL MANCERA 1542 DEPTO E 401           EJE 2     EJE 2 1542
Eje 8 SUR AV POPOCATEPETL, 415 Dpto. D 907                 EJE 8    EJE 8 415
EJE 1 OTE AV ANDRES MOLINA ENRIQUEZ, 1006 Estac. 36          EJE 1     EJE 1 1006
Eje 5 SUR AV SAN ANTONIO, 7 Edif. T 3 A, Dpto. 603              EJE 5     EJE 5 7
Insurgentes Sur 1971 Edificio Estac E2-7 2                  SUR 1971      INSURGENTES SUR 7
Anillo Periferico Sur 4863 Hab.603                     SUR 4863 603       ANILLO PERIFERICO SUR 4863
Anillo Periferico Sur 4863 Hab.603                     SUR 4863 603       ANILLO PERIFERICO SUR 4863
EJE 8 SUR CALZADA ERMITA IZTAPALAPA, 945 Edif. D, Dpto. 14               EJE 8      EJE 8 945
10 de Diciembre 123 Magdalena Mixhuca                                    10       10 DE DICIEMBRE 123
Eje 10 SUR PEDRO ENRIQUEZ UREÑA, 540 Dpto. A-501                    EJE 10        EJE 10 540
EJE 1 ORIENTE FFCC HIDALGO, 1404 Edif. TRES-C, Dpto. 208            EJE 1       EJE 1 1404
Torreon 26 Depto 301 Roma Sur                                   SUR         TORREON 26


In [ ]:
len(df_catastral)

2295465

In [ ]:
"""# --- Aquí asume que ya tienes definida la función normalizar_calle() ---

# Diccionario de pruebas: ORIGINAL -> WANTED
pruebas = {
    "Cl. SUR 128, 12": "SUR 128",
    "Sur 126 a 9": "SUR 126",
    "Sur 126-a Num.17": "SUR 126",
    "Cl. SUR 128, 22 Casa 4": "SUR 128",
    "Cl. SUR 128, 22 Dpto. 1": "SUR 128",
    "SUR 122 S/N": "SUR 122",
    "Sur 122 Esq Poniente 83": "SUR 122",
    "Sur 112 a 23 Depto 12": "SUR 112",
    "Sur 120 A-6 Depto 302-C": "SUR 120",
    "Cl. SUR 124, 3 Dpto. 3": "SUR 124",
    "SUR 114 N. 46  Depto. 303": "SUR 114",
    "Sostenes Rocha 18": "SOSTENES ROCHA 18",
    "Gral. Felix u Gomez 95": "GRAL FELIX U GOMEZ 95",
    "Sostenes Rocha Gral 10": "SOSTENES ROCHA GRAL 10",
    "Cda. 3 de Observatorio Nu.10 Depto-302": "OBSERVATORIO 10",
    "Cl. GRAL MANUEL M PLATA, 123": "GENERAL MANUEL M PLATA 123",
    "Plata Gral 110": "PLATA GRAL 110",
    "Cl. CAMINO A BELEN, 85 Edif. R, Casa 223": "CAMINO A BELEN 85",
    "Vereda del Tanque 48 Lomas de Santa Fe": "VEREDA DEL TANQUE 48",
    "Calle Estrella Fugas Mz.c Lt.13": "ESTRELLA FUGAZ",
    "Prv. RAUL ZARATE MACHUCA, 11": "RAUL ZARATE MACHUCA 11",
    "CALLE 2, 270 Edif. B, Dpto. 501": "CALLE 2 270",
    "Agua Caliente 129 Edif 2 Depto 501": "AGUA CALIENTE 129",
    "Cl. LAGO GUIJA, 72 Dpto. 710": "CALLE LAGO GUIJA 72",
    "Cl. SIERRA GORDA, 23 CONJ ENCINOS TORRE G D 10": "CALLE SIERRA GORDA 23",
    "IGNACIO ALDAMA, 188 Sec. MZ-5, CASA 28": "IGNACIO ALDAMA 188",
    "Calle 602 184": "CALLE 602 184",
    "Calle 2 Num 188 Estac 64": "CALLE 2 188",
    "ALZADA VALLEJO 1307 EST.35": "CALZADA VALLEJO 1307",
    "Cl. TENOCHTITLAN, 100 Edif. D-10, Dpto. 302": "CALLE TENOCHTITLAN 100",
    "Paseo de la Hacienda 79 Depto 401 Paseo de Taxque": "PASEO DE LA HACIENDA 79",
    "Calle 21 22 Depto. 1": "CALLE 21 22",
    "MONROVIA 806 DEPTO GH 04": "MONROVIA 806",
    "Calle 26 118": "CALLE 26 118",
    "Calle 35 Num 35": "CALLE 35 35",
    "MIGUEL BERNARD, 399 Edif. B, Dpto. 36": "MIGUEL BERNARD 399",
    "Isabel La Catolica 134 Edif C Depto 03": "ISABEL LA CATOLICA 134",
    "Cl. DR NAVARRO, 60 Edif. A, Ent. M-2, Dpto. 303": "CALLE DOCTOR NAVARRO 60",
    "Rancho San Lorenzo 81 Depto 302 a": "RANCHO SAN LORENZO 81",
    "Cl. DR JOSE MARIA VERTIZ, 101 DEPTO.407": "CALLE DOCTOR JOSE MARIA VERTIZ 101",
}

# Validación
for original, esperado in pruebas.items():
    obtenido = normalizar_calle(original)
    if obtenido == esperado:
        print(f"✅ {original} -> {obtenido}")
    else:
        print(f"❌ {original}\n   Esperado: {esperado}\n   Obtenido: {obtenido}\n")

"""

❌ Cl. SUR 128, 12
   Esperado: SUR 128
   Obtenido: CALLE SUR 128 12

❌ Sur 126 a 9
   Esperado: SUR 126
   Obtenido: SUR 126 A 9

❌ Sur 126-a Num.17
   Esperado: SUR 126
   Obtenido: SUR 126 A 17

❌ Cl. SUR 128, 22 Casa 4
   Esperado: SUR 128
   Obtenido: CALLE SUR 128 22 4

❌ Cl. SUR 128, 22 Dpto. 1
   Esperado: SUR 128
   Obtenido: CALLE SUR 128 22 DPTO 1

❌ SUR 122 S/N
   Esperado: SUR 122
   Obtenido: SUR 122 S

❌ Sur 122 Esq Poniente 83
   Esperado: SUR 122
   Obtenido: SUR 122 ESQ PONIENTE 83

❌ Sur 112 a 23 Depto 12
   Esperado: SUR 112
   Obtenido: SUR 112 A 23 12

❌ Sur 120 A-6 Depto 302-C
   Esperado: SUR 120
   Obtenido: SUR 120 A 6 302 C

❌ Cl. SUR 124, 3 Dpto. 3
   Esperado: SUR 124
   Obtenido: CALLE SUR 124 3 DPTO 3

❌ SUR 114 N. 46  Depto. 303
   Esperado: SUR 114
   Obtenido: SUR 114 46 303

✅ Sostenes Rocha 18 -> SOSTENES ROCHA 18
❌ Gral. Felix u Gomez 95
   Esperado: GRAL FELIX U GOMEZ 95
   Obtenido: GENERAL FELIX U GOMEZ 95

❌ Sostenes Rocha Gral 10
   Esperado: S

In [ ]:
import re
import unicodedata

def _strip_accents(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

def normalizar_calle(direccion: str) -> str:
    """
    Versión consolidada:
    - Normaliza mayúsculas y quita acentos.
    - Normaliza abreviaturas (GRAL se mantiene GRAL).
    - Elimina ruido (DEPTO, EDIF, CASA, etc.).
    - Conserva el número principal y, si existe y aparece antes de un stopword,
      conserva también el primer número exterior (ej: "SUR 128, 12" -> "SUR 128 12").
    - Elimina sufijos tipo "A", "A-6", "BIS" tras números.
    - Quita "CALLE" inicial cuando es redundante con SUR/NORTE/PONIENTE/ORIENTE/EJE.
    """
    if not isinstance(direccion, str):
        return ""

    # 1) normalizar base
    t = _strip_accents(direccion).upper()
    # conservar comas para detección de exterior number; diascard otros símbolos salvo @,#,-
    t = re.sub(r"[^\w\s@#\-,]", " ", t)   # \w incluye dígitos y letras sin acentos ya
    t = re.sub(r"\s+", " ", t).strip()

    # 2) normalizar abreviaturas (no expandir GRAL a GENERAL)
    abrevs = {
        r"\bCL\b": "CALLE",
        r"\bCL\.": "CALLE",
        r"\bAVD\b": "AVENIDA",
        r"\bAV\b": "AVENIDA",
        r"\bCALZ\b": "CALZADA",
        r"\bPTE\b": "PONIENTE",
        r"\bNTE\b": "NORTE",
        r"\bOTE\b": "ORIENTE",
        r"\bCDA\b": "CDA",
        r"\bPRV\b": "PRV",
        r"\bGRAL\b": "GRAL",
        r"\bDR\b": "DR",
        r"\bNUM\b": "",
        r"\bNO\b": "",
        r"\bN\b": "",
    }
    for pat, rep in abrevs.items():
        t = re.sub(pat, rep, t)
    t = re.sub(r"\s+", " ", t).strip()

    # 3) tokenizar (mantenemos comas como tokens para detectar exterior number)
    # separar comas como tokens
    t = t.replace(",", " , ")
    tokens = [tok for tok in t.split() if tok != ""]

    # sets
    street_types = {"CALLE","AVENIDA","CALZADA","PASEO","EJE","SUR","NORTE","ORIENTE","PONIENTE","CDA","PRV",
                    "VEREDA","RANCHO","PICO","BOULEVARD","BLVD","FERROCARRIL"}
    stopwords = {"DEPTO","DPTO","DEPARTAMENTO","CASA","EDIF","EDIFICIO","TORRE","ENT","ENTRADA","SEC","SECCION",
                 "MZ","MANZANA","LT","LOTE","COND","CONJ","CONJUNTO","GRUPO","PRIVADA","INT","INTERIOR","ESQ",
                 "ESTAC","EST","ALMACENES","ANDADOR","PISO","AGRUP","COL","COLONIA"}

    # helpers
    def is_num(tok):
        return bool(re.match(r"^\d+$", tok))
    def leading_digits(tok):
        m = re.match(r"^(\d+)", tok)
        return m.group(1) if m else None
    def is_letter_suffix(tok):   # 'A', 'BIS', 'B', 'C-6', etc.
        return bool(re.match(r"^[A-Z]{1,3}(-[A-Z0-9]+)?$", tok))

    # 4) buscar índice de tipo de vialidad (si existe)
    idx_st = None
    for i, tok in enumerate(tokens):
        if tok in street_types:
            idx_st = i
            break

    # función para construir resultado dada una posición de inicio y primer número index
    def build_from(idx_name, idx_num):
        # nombre: tokens desde idx_name hasta idx_num-1 (incluir tipo)
        name_parts = [tok for tok in tokens[idx_name:idx_num] if tok != ","]
        main_num = leading_digits(tokens[idx_num]) or tokens[idx_num]
        # limpio sufijos tipo 'A' pegados al numero (114A -> 114), o token next letter
        # (pero no tocamos números exteriores aún)
        # buscar primer número exterior después de idx_num y antes de cualquier stopword
        ext = None
        j = idx_num + 1
        while j < len(tokens):
            tok = tokens[j]
            if tok == ",":
                j += 1
                continue
            if tok in stopwords:
                break
            if is_num(tok):
                ext = tok
                break
            # si token es single letter immediately after main number (suffix) skip it
            if is_letter_suffix(tok):
                # skip (we already remove suffixes)
                j += 1
                continue
            # anything else -> stop searching for ext
            break
        # construir res
        res_parts = name_parts[:]
        # si name_parts empieza con CALLE seguido de SUR/NORTE/... y queremos eliminar CALLE redundante:
        if len(res_parts) >= 2 and res_parts[0] == "CALLE" and res_parts[1] in {"SUR","NORTE","PONIENTE","ORIENTE","EJE"}:
            res_parts = res_parts[1:]  # eliminar 'CALLE' inicial redundante
        res_parts.append(main_num)
        if ext:
            res_parts.append(ext)
        # eliminar duplicados num at the end (e.g., "... 35 35")
        if len(res_parts) >= 2 and re.fullmatch(r"\d+", res_parts[-1]) and res_parts[-1] == res_parts[-2]:
            res_parts = res_parts[:-1]
        return " ".join(res_parts)

    # 5) Si hay tipo de vialidad
    if idx_st is not None:
        # buscar primer índice numérico después de idx_st
        idx_num = None
        for k in range(idx_st+1, len(tokens)):
            tok = tokens[k]
            if is_num(tok):
                idx_num = k
                break
            # si token looks like "114-A" or "114A" treat as numeric with suffix
            ld = leading_digits(tok)
            if ld:
                idx_num = k
                break
            # otherwise continue collecting name words
        if idx_num is not None:
            # limpiar sufijo si tokens[idx_num] tiene attached letters, keep only digits
            if not is_num(tokens[idx_num]):
                d = leading_digits(tokens[idx_num])
                if d:
                    tokens[idx_num] = d
            return build_from(idx_st, idx_num)
        else:
            # no number found: return name from idx_st until a stopword
            name_end = len(tokens)
            for j in range(idx_st+1, len(tokens)):
                if tokens[j] in stopwords:
                    name_end = j; break
            res = " ".join([tok for tok in tokens[idx_st:name_end] if tok != ","])
            # eliminar "CALLE" redundante
            if res.startswith("CALLE ") and res.split()[1] in {"SUR","NORTE","PONIENTE","ORIENTE","EJE"}:
                res = " ".join(res.split()[1:])
            return res.strip()

    # 6) si no hay tipo de vialidad, fallback: tomar primer número como main, conservar possible next number
    idx_num = None
    for i, tok in enumerate(tokens):
        if is_num(tok) or leading_digits(tok):
            idx_num = i
            if not is_num(tok) and leading_digits(tok):
                tokens[i] = leading_digits(tok)
            break
    if idx_num is not None:
        # name part is tokens[:idx_num]
        # find ext number before a stopword
        ext = None
        j = idx_num + 1
        while j < len(tokens):
            if tokens[j] in stopwords:
                break
            if is_num(tokens[j]):
                ext = tokens[j]; break
            if tokens[j] == ",":
                j += 1; continue
            if is_letter_suffix(tokens[j]):
                j += 1; continue
            break
        name = " ".join([tok for tok in tokens[:idx_num] if tok != ","])
        if name == "":
            # no name, just return number (rare)
            res = tokens[idx_num] if ext is None else f"{tokens[idx_num]} {ext}"
            return res
        res = name + " " + tokens[idx_num]
        if ext:
            res = res + " " + ext
        # remove duplicate trailing numbers
        res = re.sub(r"\b(\d+)\s+\1\b", r"\1", res)
        return res.strip()

    # 7) fallback: devolver texto limpio sin comas ni multiples espacios
    res = " ".join([tok for tok in tokens if tok != ","])
    res = re.sub(r"\s+", " ", res).strip()
    return res


In [ ]:
import unicodedata
import re
import pandas as pd

# helper: quitar acentos y normalizar espacios/mayúsculas
def strip_accents(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s2 = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    s2 = re.sub(r"\s+", " ", s2).strip()
    return s2.upper()

# --- Diccionario de pruebas (inserta aquí todos tus casos) ---
pruebas = {
    "Cl. SUR 128, 12": "SUR 128 12",
    "Sur 126 a 9": "SUR 126 9",
    "Sur 126-a Num.17": "SUR 126 17",
    "Cl. SUR 128, 22 Casa 4": "SUR 128 22",
    "Cl. SUR 128, 22 Dpto. 1": "SUR 128 22",
    "SUR 122 S/N": "SUR 122",
    "Sur 122 Esq Poniente 83": "SUR 122",
    "Sur 112 a 23 Depto 12": "SUR 112 23",
    "Sur 120 A-6 Depto 302-C": "SUR 120",
    "Cl. SUR 124, 3 Dpto. 3": "SUR 124 3",
    "SUR 114 N. 46  Depto. 303": "SUR 114 46",
    "Sostenes Rocha 18": "SOSTENES ROCHA 18",
    "Gral. Felix u Gomez 95": "GRAL FELIX U GOMEZ 95",
    "Sostenes Rocha Gral 10": "SOSTENES ROCHA GRAL 10",
    "Cda. 3 de Observatorio Nu.10 Depto-302": "OBSERVATORIO 10",
    "Cl. GRAL MANUEL M PLATA, 123": "CALLE GRAL MANUEL M PLATA 123",
    "Plata Gral 110": "PLATA GRAL 110",
    "Cl. CAMINO A BELEN, 85 Edif. R, Casa 223": "CALLE CAMINO A BELEN 85",
    "Vereda del Tanque 48 Lomas de Santa Fe": "VEREDA DEL TANQUE 48",
    "Calle Estrella Fugas Mz.c Lt.13": "CALLE ESTRELLA FUGAZ",
    "Prv. RAUL ZARATE MACHUCA, 11": "PRV RAUL ZARATE MACHUCA 11",
    "CALLE 2, 270 Edif. B, Dpto. 501": "CALLE 2 270",
    "Agua Caliente 129 Edif 2 Depto 501": "AGUA CALIENTE 129",
    "Cl. LAGO GUIJA, 72 Dpto. 710": "CALLE LAGO GUIJA 72",
    "Cl. SIERRA GORDA, 23 CONJ ENCINOS TORRE G D 10": "CALLE SIERRA GORDA 23",
    "IGNACIO ALDAMA, 188 Sec. MZ-5, CASA 28": "IGNACIO ALDAMA 188",
    "Calle 602 184": "CALLE 602 184",
    "Calle 2 Num 188 Estac 64": "CALLE 2 188",
    "CALZADA VALLEJO 1307 EST.35": "CALZADA VALLEJO 1307",
    "Cl. TENOCHTITLAN, 100 Edif. D-10, Dpto. 302": "CALLE TENOCHTITLAN 100",
    "Paseo de la Hacienda 79 Depto 401 Paseo de Taxque": "PASEO DE LA HACIENDA 79",
    "Calle 21 22 Depto. 1": "CALLE 21 22",
    "MONROVIA 806 DEPTO GH 04": "MONROVIA 806",
    "Calle 26 118": "CALLE 26 118",
    "MIGUEL BERNARD, 399 Edif. B, Dpto. 36": "MIGUEL BERNARD 399",
    "Isabel La Catolica 134 Edif C Depto 03": "ISABEL LA CATOLICA 134",
    "Cl. DR NAVARRO, 60 Edif. A, Ent. M-2, Dpto. 303": "CALLE DR NAVARRO 60",
    "Rancho San Lorenzo 81 Depto 302 a": "RANCHO SAN LORENZO 81",
    "Cl. DR JOSE MARIA VERTIZ, 101 DEPTO.407": "CALLE DR JOSE MARIA VERTIZ 101",
}

# ------------- Validación -------------
resultados = []
for original, esperado in pruebas.items():
    obtenido = normalizar_calle(original)              # tu función debe estar definida
    esperado_norm = strip_accents(esperado)
    obtenido_norm = strip_accents(obtenido)
    ok = (obtenido_norm == esperado_norm)
    resultados.append({
        "original": original,
        "expected": esperado,
        "obtained": obtenido,
        "ok": ok
    })

df = pd.DataFrame(resultados)
total = len(df)
ok_count = df["ok"].sum()
failed = total - ok_count
pct = ok_count/total*100

print(f"Total: {total} | OK: {ok_count} | Failed: {failed} | Acierto: {pct:.1f}%\n")

# Mostrar resultados fallidos directamente
mismatches = df[df["ok"] == False].reset_index(drop=True)
if mismatches.shape[0] == 0:
    print("✅ Todos los casos pasaron")
else:
    print("❌ Casos fallidos:\n")

    for i, row in mismatches.iterrows():
        print(f"ORIGINAL : {row['original']}")
        print(f"ESPERADO : {row['expected']}")
        print(f"OBTENIDO : {row['obtained']}\n")


Total: 39 | OK: 37 | Failed: 2 | Acierto: 94.9%

❌ Casos fallidos:

ORIGINAL : Cda. 3 de Observatorio Nu.10 Depto-302
ESPERADO : OBSERVATORIO 10
OBTENIDO : CDA 3

ORIGINAL : Calle Estrella Fugas Mz.c Lt.13
ESPERADO : CALLE ESTRELLA FUGAZ
OBTENIDO : CALLE ESTRELLA FUGAS MZ C LT 13



Ultima version del código

In [ ]:
import re

def normalizar_calle(calle: str) -> str:
    if not calle:
        return ""

    # --- 1. Normalización básica ---
    txt = calle.upper()
    txt = re.sub(r"[^\w\sÁÉÍÓÚÑ#@]", " ", txt)  # elimina puntuación rara
    txt = re.sub(r"\s+", " ", txt).strip()

    # --- 2. Diccionario de abreviaturas ---
    abrevs = {
        r"\bCL\b": "CALLE",
        r"\bCLL?\b": "CALLE",
        r"\bCDA\b": "CDA",  # solo mantenemos si no hay calle clara
        r"\bPRV\.?\b": "PRV",
        r"\bAVD?\.?\b": "AVENIDA",
        r"\bAV\b": "AV",
        r"\bCALZ\.?\b": "CALZADA",
        r"\bEJE\b": "EJE",
        r"\bPTE\b": "PONIENTE",
        r"\bNTE\b": "NORTE",
        r"\bN\b": "NORTE",
        r"\bSUR\b": "SUR",
        r"\bOTE\b": "ORIENTE",
        r"\bORIENTE\b": "ORIENTE",
        r"\bGRAL\.?\b": "GRAL",
        r"\bDOCTOR\b": "DR",
    }
    for pat, repl in abrevs.items():
        txt = re.sub(pat, repl, txt)

    # --- 3. Correcciones específicas (errores OCR / humanos) ---
    correcciones = {
        "FUGAS": "FUGAZ",
    }
    for k, v in correcciones.items():
        txt = txt.replace(k, v)

    # --- 4. Normalizar fechas como nombres de calle ---
    txt = re.sub(r"\b(\d{1,2})\s*O\s*DE\s+NOVIEMBRE\b", r"20 DE NOVIEMBRE", txt)
    txt = re.sub(r"\b(\d{1,2})\s*DE\s+([A-ZÁÉÍÓÚ]+)", lambda m: m.group(0), txt)

    # --- 5. Eliminar referencias NO deseadas ---
    txt = re.sub(r"\b(EDIF\.?|EDIFICIO|DPTO\.?|DEPTO|CASA|COND\.?|CONJ\.?|ESTAC\.?|ENT\.?|TORRE|MZ\.?|LT\.?|HAB\.?)\b.*", "", txt)

    # --- 6. Truncar en colonias ---
    txt = re.sub(r"\b(COL\.?|COLONIA|FRACC\.?|FRACCIONAMIENTO|BARRIO|LOMAS|ROMA SUR|MIXHUCA|SANTA FE)\b.*", "", txt)

    # --- 7. Manejo especial de prefijos ---
    # Si hay "CDA" o "PRV" al inicio y después viene un nombre, eliminarlo
    txt = re.sub(r"^(CDA|PRV)\s+\d*\s*DE?\s*", "", txt)

    # --- 8. Números sueltos con guiones o letras ---
    txt = re.sub(r"\s+[A-Z]-?\d+\b", "", txt)  # limpia "A-6", "C-9" etc.
    txt = re.sub(r"\s+[A-Z]\b", "", txt)      # limpia letras sueltas

    # --- 9. Preservar "SUR / NORTE / ORIENTE / PONIENTE" dentro de nombre principal ---
    # Cortar solo si están después de un número
    txt = re.sub(r"(\d+)\s+(SUR|NORTE|ORIENTE|PONIENTE)\b.*", r"\1", txt)

    # --- 10. Limpieza final ---
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt
